In [1]:
%matplotlib inline
import sparse_ternary_networks.Tools as T
import numpy as np
import sparse_ternary_networks.MLSTC as MLSTC
from scipy.linalg import eigh
from scipy.linalg import toeplitz
import matplotlib.pyplot as plt
import RRQ.Tools as Tools

In [2]:
# Initializing the parameters:
n = 512      # dimension
m = 512       # dimension of the codes
N_train = 10000     # number of samples
N_test = 10000     # number of samples
k = 5         # number of non-zero elements of the STC at each stage.
L = 70        # number of layer units of the network.

In [3]:
numRep = 1
RhoList = [0.90]
#RhoList = [0,0.5,0.99]
MethodList = ['MLSTC']
#MethodList = ['RQ']
for RhoInd,Rho in enumerate(RhoList):
    for Method in MethodList:
        exec('dist_' + Method + '_Rho' + str(int(Rho*100)) + '_train = np.zeros((numRep,L+1))')
        exec('dist_' + Method + '_Rho' + str(int(Rho*100)) + '_test = np.zeros((numRep,L+1))')
        #
        exec('rate_' + Method + '_Rho' + str(int(Rho*100)) + ' = np.zeros((numRep,L+1))')

In [4]:
for RepInd in range(numRep):
    print('.......................................  Repetition = ',RepInd + 1)
    for RhoInd,Rho in enumerate(RhoList):
        print('....................  Rho = ',Rho)
        F_train,F_test = Tools.data_generator(n,N_train,N_test,'AR1',SourceParam=Rho)
        norm_train = np.divide(np.linalg.norm(F_train) ** 2, np.prod(F_train.shape))
        norm_test = np.divide(np.linalg.norm(F_test) ** 2, np.prod(F_test.shape))
        for MethodInd,Method in enumerate(MethodList):
            print(' ############   ' + Method + '     A L G O R I T H M   ################')
            
                ###############################################
            # training:
            obj_train = MLSTC.BaseLearner(k,L,m=m)
            _, _, _ = obj_train.run(F_train)
            exec('dist_' + Method + '_Rho' + str(int(Rho*100)) + '_train[RepInd,:] = obj_train.distortion/norm_train')
            exec('rate_' + Method + '_Rho' + str(int(Rho*100)) + '[RepInd,:] = obj_train.rate')
            # testing:
            obj_test = MLSTC.fwdPass(obj_train.params,k,ternaryProbMap=obj_train.prob_z)
            _,_,_ = obj_test.run(F_test)
            exec('dist_' + Method + '_Rho' + str(int(Rho*100)) + '_test[RepInd,:] = obj_test.distortion/norm_test')

.......................................  Repetition =  1
....................  Rho =  0.9
 ############   MLSTC     A L G O R I T H M   ################
 ################## Starting to learn network parameters:  ##################
layer-units:
** 1 **** 2 **** 3 **** 4 **** 5 **** 6 **** 7 **** 8 **** 9 **** 10 **** 11 **** 12 **** 13 **** 14 **** 15 **
** 16 **** 17 **** 18 **** 19 **** 20 **** 21 **** 22 **** 23 **** 24 **** 25 **** 26 **** 27 **** 28 **** 29 **** 30 **
** 31 **** 32 **** 33 **** 34 **** 35 **** 36 **** 37 **** 38 **** 39 **** 40 **** 41 **** 42 **** 43 **** 44 **** 45 **
** 46 **** 47 **** 48 **** 49 **** 50 **** 51 **** 52 **** 53 **** 54 **** 55 **** 56 **** 57 **** 58 **** 59 **** 60 **
** 61 **** 62 **** 63 **** 64 **** 65 **** 66 **** 67 **** 68 **** 69 **** 70 **
Finished learning network parameters:
 Running the network:
 ##################  Running the network:  ##################
layer-units:
** 1 **** 2 **** 3 **** 4 **** 5 **** 6 **** 7 **** 8 **** 9 ****

In [5]:
curve_dict = {}
for RhoInd,Rho in enumerate(RhoList):
    for MethodInd,Method in enumerate(MethodList):
        exec('x = np.mean(rate_' +  Method + '_Rho' + str(int(Rho*100)) + ',axis=0)')
        exec('y = np.mean(dist_' +  Method + '_Rho' + str(int(Rho*100)) + '_train,axis=0)')
        exec("curve_dict['Rho" + str(int(Rho*100))+ '_n' + str(n)+ '_m' + str(m) +"_DR_"+ Method + "_train'] = (x,y)") 
        exec('y = np.mean(dist_' +  Method + '_Rho' + str(int(Rho*100)) + '_test,axis=0)')
        exec("curve_dict['Rho" + str(int(Rho*100))+ '_n' + str(n)+ '_m' + str(m) +"_DR_"+ Method + "_test'] = (x,y)") 

In [6]:
# Saving the results for PGFplott:
PGF_path = 'PGF/dat/'
ExpName = 'SyntheticAR1_STC'
#
for curve_name, curve_xy in curve_dict.items():
    fname = ExpName + '_' + curve_name + '.dat'
    print(fname)
    x = curve_xy[0].astype(float).reshape(-1)
    y = curve_xy[1].astype(float).reshape(-1)
    np.savetxt(PGF_path + fname, np.transpose([x,y]), fmt='%8f', delimiter='   ')

SyntheticAR1_STC_Rho90_n512_m512_DR_MLSTC_train.dat
SyntheticAR1_STC_Rho90_n512_m512_DR_MLSTC_test.dat
